In [95]:
# import the required modules
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import json

In [8]:
# get the environment variable
load_dotenv()

True

In [9]:
# specify the city and get the appid
city  = "Amsterdam"
appid = os.environ.get('appid')

In [11]:
# define the endpoint and get the response
endpoint = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={appid}&units=metric"


response = requests.get(endpoint)

In [17]:
# respons.json()
weather_dict = json.loads(response.text)
weather_dict


{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1702026000,
   'main': {'temp': 2.74,
    'feels_like': -0.04,
    'temp_min': 2.74,
    'temp_max': 3.11,
    'pressure': 1004,
    'sea_level': 1004,
    'grnd_level': 1005,
    'humidity': 90,
    'temp_kf': -0.37},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 75},
   'wind': {'speed': 2.79, 'deg': 165, 'gust': 6.39},
   'visibility': 10000,
   'pop': 0.28,
   'rain': {'3h': 0.23},
   'sys': {'pod': 'd'},
   'dt_txt': '2023-12-08 09:00:00'},
  {'dt': 1702036800,
   'main': {'temp': 3.5,
    'feels_like': 1.05,
    'temp_min': 3.5,
    'temp_max': 5.02,
    'pressure': 1005,
    'sea_level': 1005,
    'grnd_level': 1005,
    'humidity': 92,
    'temp_kf': -1.52},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': {'all': 83},
   'wind': {'speed': 2.58, 'deg': 182, 'gust': 5.99}

In [18]:
# show the highest level keys
weather_dict.keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [21]:
# show the information of the city
weather_dict['city']

{'id': 2759794,
 'name': 'Amsterdam',
 'coord': {'lat': 52.374, 'lon': 4.8897},
 'country': 'NL',
 'population': 2122311,
 'timezone': 3600,
 'sunrise': 1702021001,
 'sunset': 1702049276}

In [35]:
# show the list of predictions
weather_dict['list'][0]['main']['temp']

2.74

In [39]:
weather_dict['list'][39]['main']['temp']

8.45

In [88]:
# collect the data from the weather dict
weather_temp = []
weather_feels_like = []
weather_rain_mm = []
weather_description = []
weather_dt_txt = []
weather_wind_speed = []


# iterate over the predictions in the list
for i, prediction in enumerate(weather_dict['list']):
    weather_temp.append(prediction['main']['temp'])
    weather_feels_like.append(prediction['main']['feels_like'])
    if 'rain' in prediction.keys():
        weather_rain_mm.append(prediction['rain']['3h'])
    else:
        weather_rain_mm.append(0)
    weather_description.append(prediction['weather'][0]['description'])
    weather_dt_txt.append(prediction['dt_txt'])
    weather_wind_speed.append(weather_dict['list'][i]['wind']['speed'])


In [89]:
weather_wind_speed[:5]

[2.79, 2.58, 3, 3.48, 4.84]

In [76]:
# list comprehension
freeze_non_freeze = [x for x in weather_temp if x < 5]
freeze_non_freeze

[2.74, 3.5]

In [90]:
# dict comprehension
{   dt:{"temperature": temp,
        "description": description,
        "feels_like": feels_like,
        "rain": rain,
        "wind_speed": wind_speed
        }
 
    for description, temp, dt, feels_like, rain, wind_speed
    in zip(weather_description, weather_temp, 
           weather_dt_txt, weather_feels_like, weather_rain_mm, weather_wind_speed)
}

{'2023-12-08 09:00:00': {'temperature': 2.74,
  'description': 'light rain',
  'feels_like': -0.04,
  'rain': 0.23,
  'wind_speed': 2.79},
 '2023-12-08 12:00:00': {'temperature': 3.5,
  'description': 'broken clouds',
  'feels_like': 1.05,
  'rain': 0,
  'wind_speed': 2.58},
 '2023-12-08 15:00:00': {'temperature': 5.79,
  'description': 'overcast clouds',
  'feels_like': 3.45,
  'rain': 0,
  'wind_speed': 3},
 '2023-12-08 18:00:00': {'temperature': 6.17,
  'description': 'broken clouds',
  'feels_like': 3.6,
  'rain': 0,
  'wind_speed': 3.48},
 '2023-12-08 21:00:00': {'temperature': 7.08,
  'description': 'broken clouds',
  'feels_like': 4.01,
  'rain': 0,
  'wind_speed': 4.84},
 '2023-12-09 00:00:00': {'temperature': 7.45,
  'description': 'overcast clouds',
  'feels_like': 4.35,
  'rain': 0,
  'wind_speed': 5.13},
 '2023-12-09 03:00:00': {'temperature': 6.96,
  'description': 'light rain',
  'feels_like': 3.68,
  'rain': 0.13,
  'wind_speed': 5.24},
 '2023-12-09 06:00:00': {'temperat

In [93]:
# or, create a Pandas DataFrame

weather_df = pd.DataFrame(
    {
        "date": weather_dt_txt,
        "temperature": weather_temp,
        "feels_like":  weather_feels_like,
        "rain_mm": weather_rain_mm,
        "weather_wind_speed": weather_wind_speed,
        "description": weather_description
    }
)

In [94]:
# describe
weather_df['temperature'].describe()

count    40.000000
mean      8.110250
std       1.656365
min       2.740000
25%       7.365000
50%       8.465000
75%       9.222500
max      10.840000
Name: temperature, dtype: float64

# Assignments (2)

## Assignment 1

Add an additional column to the weather_df dataframe which indicates if it is freezing or not.

In [99]:
weather_df['freeze'] = np.where(weather_df['temperature'] > 0, "nee", "ja")
weather_df['freeze'].value_counts()

freeze
nee    40
Name: count, dtype: int64

## Assignment 2

Create a column that displays the temperatur in Fahrenheid

In [102]:
weather_df['temp_fahrenheit'] = weather_df['temperature'] * 1.8 + 31
weather_df[['temperature', 'temp_fahrenheit']]

,temperature,temp_fahrenheit
0,2.74,35.932
1,3.50,37.300
2,5.79,41.422
3,6.17,42.106
4,7.08,43.744
5,7.45,44.410
6,6.96,43.528
7,6.67,43.006
8,6.06,41.908
9,7.11,43.798


## Assignment 3

Filter the data frame which will only contain rows with a temperature higher than 7 degrees celcius

In [107]:
# with brackets
weather_df_subset = weather_df[weather_df['temperature'] > 7]
weather_df_subset.head()

,date,temperature,feels_like,rain_mm,weather_wind_speed,description,freeze,temp_fahrenheit
4,2023-12-08 21:00:00,7.08,4.01,0.00,4.84,broken clouds,nee,43.744
5,2023-12-09 00:00:00,7.45,4.35,0.00,5.13,overcast clouds,nee,44.410
9,2023-12-09 12:00:00,7.11,3.11,1.79,7.37,light rain,nee,43.798
10,2023-12-09 15:00:00,7.46,3.29,3.56,8.29,moderate rain,nee,44.428
11,2023-12-09 18:00:00,10.84,10.28,1.82,8.36,light rain,nee,50.512


In [106]:
# with .query
weather_df_subset = weather_df.query("temperature > 7")
weather_df_subset.head()

,date,temperature,feels_like,rain_mm,weather_wind_speed,description,freeze,temp_fahrenheit
4,2023-12-08 21:00:00,7.08,4.01,0.00,4.84,broken clouds,nee,43.744
5,2023-12-09 00:00:00,7.45,4.35,0.00,5.13,overcast clouds,nee,44.410
9,2023-12-09 12:00:00,7.11,3.11,1.79,7.37,light rain,nee,43.798
10,2023-12-09 15:00:00,7.46,3.29,3.56,8.29,moderate rain,nee,44.428
11,2023-12-09 18:00:00,10.84,10.28,1.82,8.36,light rain,nee,50.512


In [109]:
# specify the rains
rains = ['light rain', 'moderate rain', 'heavy rain']

# 
weather_df.query("description == @rains")

,date,temperature,feels_like,rain_mm,weather_wind_speed,description,freeze,temp_fahrenheit
0,2023-12-08 09:00:00,2.74,-0.04,0.23,2.79,light rain,nee,35.932
6,2023-12-09 03:00:00,6.96,3.68,0.13,5.24,light rain,nee,43.528
9,2023-12-09 12:00:00,7.11,3.11,1.79,7.37,light rain,nee,43.798
10,2023-12-09 15:00:00,7.46,3.29,3.56,8.29,moderate rain,nee,44.428
11,2023-12-09 18:00:00,10.84,10.28,1.82,8.36,light rain,nee,50.512
12,2023-12-09 21:00:00,9.16,5.27,0.16,9.23,light rain,nee,47.488
13,2023-12-10 00:00:00,8.26,4.04,0.22,9.42,light rain,nee,45.868
14,2023-12-10 03:00:00,6.91,2.09,1.18,10.10,light rain,nee,43.438
15,2023-12-10 06:00:00,8.48,3.90,2.78,11.39,light rain,nee,46.264
16,2023-12-10 09:00:00,8.27,3.86,0.22,10.25,light rain,nee,45.886


In [110]:
# specify the rains
rains = ['light rain', 'moderate rain', 'heavy rain']

# 
weather_df[weather_df['description'].isin(rains)]

,date,temperature,feels_like,rain_mm,weather_wind_speed,description,freeze,temp_fahrenheit
0,2023-12-08 09:00:00,2.74,-0.04,0.23,2.79,light rain,nee,35.932
6,2023-12-09 03:00:00,6.96,3.68,0.13,5.24,light rain,nee,43.528
9,2023-12-09 12:00:00,7.11,3.11,1.79,7.37,light rain,nee,43.798
10,2023-12-09 15:00:00,7.46,3.29,3.56,8.29,moderate rain,nee,44.428
11,2023-12-09 18:00:00,10.84,10.28,1.82,8.36,light rain,nee,50.512
12,2023-12-09 21:00:00,9.16,5.27,0.16,9.23,light rain,nee,47.488
13,2023-12-10 00:00:00,8.26,4.04,0.22,9.42,light rain,nee,45.868
14,2023-12-10 03:00:00,6.91,2.09,1.18,10.10,light rain,nee,43.438
15,2023-12-10 06:00:00,8.48,3.90,2.78,11.39,light rain,nee,46.264
16,2023-12-10 09:00:00,8.27,3.86,0.22,10.25,light rain,nee,45.886


In [115]:
# should contain the term 'cloud'
term_cloud = 'cloud'

# 
weather_df.query("description.str.contains(@term_cloud) & temperature < 7", engine="python")

,date,temperature,feels_like,rain_mm,weather_wind_speed,description,freeze,temp_fahrenheit
1,2023-12-08 12:00:00,3.50,1.05,0.0,2.58,broken clouds,nee,37.300
2,2023-12-08 15:00:00,5.79,3.45,0.0,3.00,overcast clouds,nee,41.422
3,2023-12-08 18:00:00,6.17,3.60,0.0,3.48,broken clouds,nee,42.106
7,2023-12-09 06:00:00,6.67,3.18,0.0,5.57,overcast clouds,nee,43.006
8,2023-12-09 09:00:00,6.06,2.19,0.0,6.13,overcast clouds,nee,41.908


In [119]:
# str. for the string accessors
weather_df['description'].str.split().str[0]

0        light
1       broken
2     overcast
3       broken
4       broken
5     overcast
6        light
7     overcast
8     overcast
9        light
10    moderate
11       light
12       light
13       light
14       light
15       light
16       light
17    overcast
18    overcast
19       light
20       light
21    overcast
22    overcast
23    overcast
24    overcast
25    overcast
26       light
27       light
28       light
29       light
30       light
31    overcast
32    overcast
33       light
34       light
35       light
36       light
37       light
38       light
39       light
Name: description, dtype: object

In [120]:
# str. for the string accessors
weather_df['description'].str.replace("i", "y")

0          lyght rayn
1       broken clouds
2     overcast clouds
3       broken clouds
4       broken clouds
5     overcast clouds
6          lyght rayn
7     overcast clouds
8     overcast clouds
9          lyght rayn
10      moderate rayn
11         lyght rayn
12         lyght rayn
13         lyght rayn
14         lyght rayn
15         lyght rayn
16         lyght rayn
17    overcast clouds
18    overcast clouds
19         lyght rayn
20         lyght rayn
21    overcast clouds
22    overcast clouds
23    overcast clouds
24    overcast clouds
25    overcast clouds
26         lyght rayn
27         lyght rayn
28         lyght rayn
29         lyght rayn
30         lyght rayn
31    overcast clouds
32    overcast clouds
33         lyght rayn
34         lyght rayn
35         lyght rayn
36         lyght rayn
37         lyght rayn
38         lyght rayn
39         lyght rayn
Name: description, dtype: object

In [122]:
# create a new (higher) category for the description
weather_df['description_overall'] = weather_df['description'].str.split(" ").str[-1]
weather_df['description_overall'].value_counts()

description_overall
rain      24
clouds    16
Name: count, dtype: int64